<a href='https://colab.research.google.com/github/ishaqmarashy/red-light-traffic-violation-classification/blob/main/decision%20tree%20classifier.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

## Preprocess Data

In [216]:
import pandas as pd
import os
csv_filename1 = "data/tv_asos.csv"
df = pd.read_csv(csv_filename1)

In [217]:
df.drop(columns=['Violation Type','Charge','Article','Contributed To Accident','Article','HAZMAT','Personal Injury','Fatal'],inplace=True)


In [218]:
df

,Latitude,Longitude,Accident,Belts,Property Damage,Commercial License,Commercial Vehicle,Alcohol,Work Zone,State,...,DV,DATE,tmpf,dwpf,sknt,relh,alti,vsby,feel,skyc1
0,38.997645,-77.027105,Yes,No,No,No,No,No,No,MD,...,0,2023-04-30 03:00:00,57.919565,55.435556,3.970000,91.751778,29.741089,9.480000,57.897778,CLR
1,38.997566,-77.109724,Yes,No,No,No,No,No,No,MD,...,1,2023-05-02 12:00:00,46.962791,38.032558,7.695238,72.453721,29.453962,9.416667,44.610233,BKN
2,39.223205,-77.260313,Yes,No,Yes,No,No,No,No,MD,...,0,2023-05-16 10:00:00,55.866667,52.961905,3.707547,90.415238,29.960654,9.923077,55.869048,CLR
3,39.057987,-76.968677,Yes,No,No,No,No,No,No,MD,...,1,2023-05-17 14:00:00,64.221951,49.446341,8.872549,59.397317,29.769541,10.000000,64.229268,CLR
4,38.999282,-77.026098,Yes,No,Yes,No,No,No,No,MD,...,0,2023-05-20 10:00:00,57.889474,56.616667,2.958678,95.791481,30.064426,4.885798,57.838889,OVC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,0.000000,0.000000,Yes,No,No,No,No,No,No,MD,...,0,2013-09-17 02:00:00,58.643810,53.248095,1.634146,83.465952,30.222143,9.575000,58.635714,SCT
1782,0.000000,0.000000,Yes,No,No,No,No,No,No,MD,...,0,2013-09-25 02:00:00,53.233333,49.070000,0.512195,86.456190,29.981429,9.926829,53.203571,CLR
1783,0.000000,0.000000,Yes,No,No,No,No,No,No,MD,...,0,2013-09-19 02:00:00,55.603182,49.180455,1.697674,79.646364,30.228409,10.000000,55.557955,CLR
1784,39.144320,-77.067415,Yes,No,No,No,No,No,No,MD,...,0,2014-12-17 23:00:00,41.876923,28.556757,7.972973,57.388378,30.011538,9.921053,38.103714,CLR


https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html

https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html

In [219]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

filtered_df = df[(df['Latitude'] != 0) & (df['Longitude'] != 0) & (df['Longitude'] <= -76.9)& (df['Latitude'] >=38.93)]
montgomery_shapefile_path = './data/roads/tl_2019_24031_roads.shp'
montgomery_gdf = gpd.read_file(montgomery_shapefile_path)

geometry = [Point(xy) for xy in zip(filtered_df['Longitude'], filtered_df['Latitude'])]
geo_points = gpd.GeoDataFrame(filtered_df, geometry=geometry)

fig, ax = plt.subplots(figsize=(10, 10))

montgomery_gdf.plot(ax=ax, color='lightblue', edgecolor='black', zorder=1)
geo_points.plot(ax=ax, marker='*', color=filtered_df['DV'].map({0: 'blue', 1: 'red'}), markersize=5, zorder=2)

legend_labels = {0: 'No DV', 1: 'DV'}
ax.legend([plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10),
           plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10)],
          [legend_labels[0], legend_labels[1]], title='Legend', loc='upper right')

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.title('Montgomery County, Maryland Accidents due to RLVs')
plt.show()


In [ ]:
df.drop(columns=['Latitude', 'Longitude'],inplace=True)

In [ ]:
df.isna().sum()

Accident              0
Belts                 0
Property Damage       0
Commercial License    0
Commercial Vehicle    0
Alcohol               0
Work Zone             0
State                 0
VehicleType           0
Year                  1
Make                  0
Model                 0
Color                 5
Race                  0
Gender                0
Driver City           0
Driver State          0
DL State              1
DV                    0
DATE                  0
tmpf                  0
dwpf                  0
sknt                  0
relh                  0
alti                  0
vsby                  0
feel                  0
skyc1                 0
dtype: int64

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer


data_for_imputation = df.copy()

categorical_imputer = SimpleImputer(strategy='most_frequent')

imputed_values = categorical_imputer.fit_transform(data_for_imputation)

imputed_df = pd.DataFrame(imputed_values,columns=df.columns)

df= imputed_df

df


,Accident,Belts,Property Damage,Commercial License,Commercial Vehicle,Alcohol,Work Zone,State,VehicleType,Year,...,DV,DATE,tmpf,dwpf,sknt,relh,alti,vsby,feel,skyc1
0,Yes,No,No,No,No,No,No,MD,02 - Automobile,2020.0,...,0,2023-04-30 03:00:00,57.919565,55.435556,3.97,91.751778,29.741089,9.48,57.897778,CLR
1,Yes,No,No,No,No,No,No,MD,02 - Automobile,2023.0,...,1,2023-05-02 12:00:00,46.962791,38.032558,7.695238,72.453721,29.453962,9.416667,44.610233,BKN
2,Yes,No,Yes,No,No,No,No,MD,02 - Automobile,2023.0,...,0,2023-05-16 10:00:00,55.866667,52.961905,3.707547,90.415238,29.960654,9.923077,55.869048,CLR
3,Yes,No,No,No,No,No,No,MD,02 - Automobile,2012.0,...,1,2023-05-17 14:00:00,64.221951,49.446341,8.872549,59.397317,29.769541,10.0,64.229268,CLR
4,Yes,No,Yes,No,No,No,No,MD,02 - Automobile,2018.0,...,0,2023-05-20 10:00:00,57.889474,56.616667,2.958678,95.791481,30.064426,4.885798,57.838889,OVC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,Yes,No,No,No,No,No,No,MD,02 - Automobile,2005.0,...,0,2013-09-17 02:00:00,58.64381,53.248095,1.634146,83.465952,30.222143,9.575,58.635714,SCT
1782,Yes,No,No,No,No,No,No,MD,02 - Automobile,1997.0,...,0,2013-09-25 02:00:00,53.233333,49.07,0.512195,86.45619,29.981429,9.926829,53.203571,CLR
1783,Yes,No,No,No,No,No,No,MD,02 - Automobile,2005.0,...,0,2013-09-19 02:00:00,55.603182,49.180455,1.697674,79.646364,30.228409,10.0,55.557955,CLR
1784,Yes,No,No,No,No,No,No,MD,02 - Automobile,1996.0,...,0,2014-12-17 23:00:00,41.876923,28.556757,7.972973,57.388378,30.011538,9.921053,38.103714,CLR


In [ ]:
df['VehicleType'].value_counts()

VehicleType
02 - Automobile              1580
05 - Light Duty Truck          90
28 - Other                     40
03 - Station Wagon             31
01 - Motorcycle                18
06 - Heavy Duty Truck          15
10 - Transit Bus                6
07 - Truck/Road Tractor         3
08 - Recreational Vehicle       1
20 - Commercial Rig             1
29 - Unknown                    1
Name: count, dtype: int64

In [ ]:
def classify_vehicle(vehicle_type):
    if any(category in vehicle_type for category in ['01 - Motorcycle']):
        return 'Light'
    elif any(category in vehicle_type for category in ['28 - Other','29 - Unknown', '03 - Station Wagon', '02 - Automobile']):
        return 'Medium'
    elif any(category in vehicle_type for category in ['05 - Light Duty Truck', '07 - Truck/Road Tractor', '08 - Recreational Vehicle',]):
        return 'Heavy'
    else:
        return 'Very Heavy'
    
df['VehicleType'] = df['VehicleType'].apply(classify_vehicle)

In [ ]:
df['Year']=df['Year'].astype(int)
df['Vehicle'] = df[[ 'Year', 'Make', 'Model']].astype(str).apply(lambda x: ', '.join(x), axis=1)
df.drop(columns=['Year', 'Make', 'Model'],inplace=True)

In [ ]:
df['Vehicle'].value_counts()

Vehicle
2014, HONDA, ACCORD         9
2004, HONDA, ACCORD         9
2006, CHEV, MALIBU          7
2010, TOYOTA, COROLLA       6
2014, TOYOTA, CAMRY         6
                           ..
2002, VOLVO, S60            1
2012, NISSAN, SENTRA 4DR    1
2009, SCION, TC             1
2003, MERCEDES, E320        1
1996, ISUZU, RODEO          1
Name: count, Length: 1384, dtype: int64

In [ ]:
df.columns

Index(['Accident', 'Belts', 'Property Damage', 'Commercial License',
       'Commercial Vehicle', 'Alcohol', 'Work Zone', 'State', 'VehicleType',
       'Color', 'Race', 'Gender', 'Driver City', 'Driver State', 'DL State',
       'DV', 'DATE', 'tmpf', 'dwpf', 'sknt', 'relh', 'alti', 'vsby', 'feel',
       'skyc1', 'Vehicle'],
      dtype='object')

In [ ]:
df

,Accident,Belts,Property Damage,Commercial License,Commercial Vehicle,Alcohol,Work Zone,State,VehicleType,Color,...,DATE,tmpf,dwpf,sknt,relh,alti,vsby,feel,skyc1,Vehicle
0,Yes,No,No,No,No,No,No,MD,Medium,WHITE,...,2023-04-30 03:00:00,57.919565,55.435556,3.97,91.751778,29.741089,9.48,57.897778,CLR,"2020, LINC, UT"
1,Yes,No,No,No,No,No,No,MD,Medium,"BLUE, LIGHT",...,2023-05-02 12:00:00,46.962791,38.032558,7.695238,72.453721,29.453962,9.416667,44.610233,BKN,"2023, MAZDA, CX-5"
2,Yes,No,Yes,No,No,No,No,MD,Medium,WHITE,...,2023-05-16 10:00:00,55.866667,52.961905,3.707547,90.415238,29.960654,9.923077,55.869048,CLR,"2023, VOLK, ATLAS"
3,Yes,No,No,No,No,No,No,MD,Medium,GRAY,...,2023-05-17 14:00:00,64.221951,49.446341,8.872549,59.397317,29.769541,10.0,64.229268,CLR,"2012, HONDA, ODYSSEY"
4,Yes,No,Yes,No,No,No,No,MD,Medium,BLACK,...,2023-05-20 10:00:00,57.889474,56.616667,2.958678,95.791481,30.064426,4.885798,57.838889,OVC,"2018, HOND, SW"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,Yes,No,No,No,No,No,No,MD,Medium,GREEN,...,2013-09-17 02:00:00,58.64381,53.248095,1.634146,83.465952,30.222143,9.575,58.635714,SCT,"2005, HOND, 4S"
1782,Yes,No,No,No,No,No,No,MD,Medium,BLACK,...,2013-09-25 02:00:00,53.233333,49.07,0.512195,86.45619,29.981429,9.926829,53.203571,CLR,"1997, MITS, 4S"
1783,Yes,No,No,No,No,No,No,MD,Medium,BLACK,...,2013-09-19 02:00:00,55.603182,49.180455,1.697674,79.646364,30.228409,10.0,55.557955,CLR,"2005, HOND, 4S"
1784,Yes,No,No,No,No,No,No,MD,Medium,WHITE,...,2014-12-17 23:00:00,41.876923,28.556757,7.972973,57.388378,30.011538,9.921053,38.103714,CLR,"1996, ISUZU, RODEO"


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

columns_to_encode =  ['State', 'VehicleType','Color', 'Race', 'Driver City',
                       'Driver State', 'DL State', 'skyc1', 'Vehicle','Accident',
                         'Belts', 'Property Damage', 'Commercial License', 
                         'Commercial Vehicle', 'Alcohol', 'Gender','Work Zone']

for column in columns_to_encode:
    df[column] =label_encoder.fit_transform(df[column])
df

,Accident,Belts,Property Damage,Commercial License,Commercial Vehicle,Alcohol,Work Zone,State,VehicleType,Color,...,DATE,tmpf,dwpf,sknt,relh,alti,vsby,feel,skyc1,Vehicle
0,0,0,0,0,0,0,0,12,2,19,...,2023-04-30 03:00:00,57.919565,55.435556,3.97,91.751778,29.741089,9.48,57.897778,2,1362
1,0,0,0,0,0,0,0,12,2,4,...,2023-05-02 12:00:00,46.962791,38.032558,7.695238,72.453721,29.453962,9.416667,44.610233,1,1382
2,0,0,1,0,0,0,0,12,2,19,...,2023-05-16 10:00:00,55.866667,52.961905,3.707547,90.415238,29.960654,9.923077,55.869048,2,1383
3,0,0,0,0,0,0,0,12,2,8,...,2023-05-17 14:00:00,64.221951,49.446341,8.872549,59.397317,29.769541,10.0,64.229268,2,946
4,0,0,1,0,0,0,0,12,2,1,...,2023-05-20 10:00:00,57.889474,56.616667,2.958678,95.791481,30.064426,4.885798,57.838889,4,1300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,0,0,0,0,0,0,0,12,2,9,...,2013-09-17 02:00:00,58.64381,53.248095,1.634146,83.465952,30.222143,9.575,58.635714,5,454
1782,0,0,0,0,0,0,0,12,2,1,...,2013-09-25 02:00:00,53.233333,49.07,0.512195,86.45619,29.981429,9.926829,53.203571,2,86
1783,0,0,0,0,0,0,0,12,2,1,...,2013-09-19 02:00:00,55.603182,49.180455,1.697674,79.646364,30.228409,10.0,55.557955,2,454
1784,0,0,0,0,0,0,0,12,2,19,...,2014-12-17 23:00:00,41.876923,28.556757,7.972973,57.388378,30.011538,9.921053,38.103714,2,58


In [ ]:
x=df.drop(columns=['DV'])
y=df['DV']

x['tmpf'] = pd.to_numeric(x['tmpf'], errors='coerce')
x['dwpf'] = pd.to_numeric(x['dwpf'], errors='coerce')
x['sknt'] = pd.to_numeric(x['sknt'], errors='coerce')
x['relh'] = pd.to_numeric(x['relh'], errors='coerce')
x['alti'] = pd.to_numeric(x['alti'], errors='coerce')
x['vsby'] = pd.to_numeric(x['vsby'], errors='coerce')
x['feel'] = pd.to_numeric(x['feel'], errors='coerce')
y = pd.to_numeric(y, errors='coerce')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.3,
                                                    random_state=42, stratify=y)

## SMOTE?


https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

In [ ]:
y.value_counts()

DV
0    1227
1     559
Name: count, dtype: int64

In [ ]:
print(x.dtypes) 
print(y.dtypes)  

Accident                int32
Belts                   int32
Property Damage         int32
Commercial License      int32
Commercial Vehicle      int32
Alcohol                 int32
Work Zone               int32
State                   int32
VehicleType             int32
Color                   int32
Race                    int32
Gender                  int32
Driver City             int32
Driver State            int32
DL State                int32
DATE                   object
tmpf                  float64
dwpf                  float64
sknt                  float64
relh                  float64
alti                  float64
vsby                  float64
feel                  float64
skyc1                   int32
Vehicle                 int32
dtype: object
int64


In [ ]:
x['DATE'] = pd.to_datetime(x['DATE']).astype('int64') // 10**9  

In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X_OS, y_OS = oversample.fit_resample(x, y)

In [ ]:
y_OS.value_counts()

DV
0    1227
1    1227
Name: count, dtype: int64

## Ensamble Implementation